In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import time

import matplotlib
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
from matplotlib import gridspec

import numpy as np
import scipy.sparse as sp
import numpy.linalg as la

from scipy import interpolate
from scipy.special import expit

from optimize.snopt7 import SNOPT_solver